# 프롬프트 (Prompt)
`02_prompt.ipynb`

- LLM에게 주는 입력(지시, 맥락, 기록)
    - 지시: '~~ 해줘'
    - 맥락: Context - 현재 지시를 위해 제공하는 추가 정보
    - 기억: Memory - 지금까지 했던 대화 내용 

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [23]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

from langchain_core.prompts import PromptTemplate

llm = ChatOpenAI(model='gpt-4.1-nano')

In [24]:
# 추후에 체인.invoke에서 {} 내부에 들어갈 말을 채워줘야 함.
template = '{country}의 수도가 어디인가요?'

# .from_template 메서드로 프롬프트 만들기
prompt = PromptTemplate.from_template(template)

# {}를 채우는 방법 (우리는 결국 chain.invoke 로 쓰게 됨)
prompt.format(country='대한민국')


'대한민국의 수도가 어디인가요?'

In [25]:
chain = prompt | llm

chain.invoke({'country': '대한민국'}).content

'대한민국의 수도는 서울입니다.'

# Partial Variable (부분 변수)
- 프롬프트에서 매개변수 기본값 사용하기

In [26]:
template = '{c1}과 {c2}의 수도는 각각 어디인가요?'

prompt = PromptTemplate(
    template=template,
    input_variables=['c1'],
    partial_variables={
        'c2': '미국'
    }
)

prompt.format(c1='한국')
# prompt.format(c2='캐나다') 

'한국과 미국의 수도는 각각 어디인가요?'

In [27]:
chain = prompt | llm
chain.invoke({'c1': 'kor', 'c2': 'us'}).content

'한국(대한민국)의 수도는 서울이고, 미국의 수도는 워싱턴 D.C.입니다.'

In [28]:
from datetime import datetime

def get_today():
    return datetime.now().strftime('%b %d')

prompt = PromptTemplate(
    template='오늘 날짜는 {today}입니다. 오늘 생일인 유명인 {n}명을 생년월일과 함께 나열해 주세요',
    input_variables=['n'],
    partial_variables={
        'today': get_today
    }
)

prompt.format(n=3)

'오늘 날짜는 Sep 01입니다. 오늘 생일인 유명인 3명을 생년월일과 함께 나열해 주세요'

In [29]:
llm = ChatOpenAI(model='gpt-5-nano')

chain = prompt | llm
# 오늘 날짜 기준
print(chain.invoke({'n': 3}).content)
print('===')
print(chain.invoke({'n': 3, 'today': 'Jan 2'}).content)

다음 두 명은 오늘 생일인 유명인으로 확인됩니다.
- Zendaya — 1996년 9월 1일
- Dr. Phil (Phil McGraw) — 1950년 9월 1일

세 번째 인물을 정확하게 확인해 세 명으로 맞춰 드리려면 웹 검색이 필요합니다. 원하시면 바로 찾아서 추가로 알려드릴게요. 어떤 분야의 인물을 원하시나요? (예: 영화/TV, 음악, 스포츠 등)
===
좋아요. 오늘이 1월 2일이라고 하셨으니, 생일이 오늘인 유명인을 생년월일과 함께 알려드리겠습니다. 다만 세 명을 확정하려면 최신 정보 확인이 필요합니다. 먼저 한 명은 확실합니다.

- Isaac Asimov — 1920년 1월 2일생

나머지 두 명은 최신 정보를 확인해 바로 추가로 찾아드릴 수 있습니다. 웹 검색을 허용해 주시겠어요? 허용해 주시면 바로 2명을 더 찾아서 목록으로 정리해 드리겠습니다.


## ChatPromptTemplate
- 채팅을 주고받는 템플릿 생성용
- 대화 목록을 LLM에게 주입
- 하나의 Chat 은 role 과 message 로 구성됨

In [30]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template('{country}의 수도는 어디?')
chat_prompt.format(country='한국')

'Human: 한국의 수도는 어디?'

In [31]:
chat_template = ChatPromptTemplate.from_messages(
    [
        # role - message
        ('system', '당신은 친절한 AI어시스트. 이름은 {name} 야.'),
        ('human', '반가워!'),
        ('ai', '무엇을 도와드릴까요?'),
        ('human', '{user_input}')
    ]
)

# format vs format_messages
messages_str = chat_template.format(name='gaida', user_input='이름이 뭐니?')
messages_cls = chat_template.format_messages(name='gaida', user_input='이름이 뭐니?')
print(messages_str)
print(messages_cls)

System: 당신은 친절한 AI어시스트. 이름은 gaida 야.
Human: 반가워!
AI: 무엇을 도와드릴까요?
Human: 이름이 뭐니?
[SystemMessage(content='당신은 친절한 AI어시스트. 이름은 gaida 야.', additional_kwargs={}, response_metadata={}), HumanMessage(content='반가워!', additional_kwargs={}, response_metadata={}), AIMessage(content='무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='이름이 뭐니?', additional_kwargs={}, response_metadata={})]


In [32]:
# 한덩어리 텍스트
llm.invoke(messages_str).content
# 실제 대화 내역으로 인지 (Langsmith 가서 확인)
llm.invoke(messages_cls).content

'제 이름은 gaida예요. 반가워요! 무엇을 도와드릴까요?'

In [33]:
chain = chat_template | llm | StrOutputParser()

chain.invoke({'name': '가이다', 'user_input': '내 이름은 장기진. 너와 나의 이름점을 봐줘'})

'좋아요! 장기진 님과 제 이름 가이다의 이름 궁합을 간단하게 재미로 계산해봤어요.\n\n결과: 약 42% 호환\n\n계산 방식 요약\n- 초성 겹침(첫 글자들): 가이다의 초성은 ㄱ, ㅇ, ㄷ이고 장기진은 ㅈ, ㄱ, ㅈ로, 공통은 ㄱ 하나 → 약 8% 정도로 반영\n- 모음 겹침: 가이다의 모음 ㅏ, ㅣ, ㅏ와 장기진의 모음 ㅏ, ㅣ, ㅣ의 공통은 ㅏ와 ㅣ 두 개 → 약 33%로 반영\n- 종성 겹침: 가이다에 종성은 없고 장기진은 ㅇ, ㄴ이므로 공통 없음 → 0%\n\n해석\n- 모음에서의 공통이 꽤 있어 서로의 분위기가 잘 맞을 수 있습니다.\n- 초성은 하나밖에 겹치지 않아 조화보다는 서로의 개성이 두드러질 수 있습니다.\n- 전체적으로는 재미삼아 보는 수준의 수치이며, 현실적인 궁합은 아닙니다.\n\n다른 방식으로도 계산해볼까요?\n- 이름의 성 분리 여부를 반영하거나, 다른 음운 규칙으로 다시 계산해드릴 수 있어요.\n- 원하시면 성/이름 전체를 다시 입력해 다른 규칙으로도 해볼게요.'

## 프롬프팅 방법
내가 원하는 답변을 구구절절 설명 없이 예시를 주고 답변의 형태를 유도
- Zero-shot prompting : 예시 없이 질문만 -> 바로 답변
- One-shot prompting : 예시 1개 + 질문 -> 모델이 예시를 모방해서 답변
- **Few-shot prompting** : 예시 여러개 + 질문 -> 예시들의 패턴을 일반화 해서 답변

In [36]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model='gpt-4.1-nano')

examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인
""",
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일
""",
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군
""",
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예
""",
    },
]

example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"

chain = prompt | llm | StrOutputParser()

res = chain.invoke({'question': question})

print(res)


이 질문에 추가 질문이 필요한가요: 예.  
추가 질문: Google이 창립된 연도는 언제인가요?  
중간 답변: Google은 1998년에 창립되었습니다.  
추가 질문: Bill Gates는 1998년에 몇 살이었나요?  
중간 답변: Bill Gates는 1955년 10월 28일생으로, 1998년에는 만 42세였습니다.  
최종 답변은: 42세


## langchain-hub
다양한 사용자들이 업로드한 프롬프트를 받아서 활용

In [35]:
from langchain import hub

prompt = hub.pull('hwchase17/react')
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [ ]:
prompt = PromptTemplate.from_template(
    '내 첫 프롬프트 Push!'

)
hub.push('')